## Scrape personal finance wiki page from reddit.

In [4]:
%pip install python-dotenv
%pip install praw

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 7.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

Python-dotenv could not parse statement starting at line 3


True

In [4]:

client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user_agent = os.getenv("REDDIT_USER_AGENT")

yPfrJvPI6bxW94SC6H44hw yiuvn_WmWNxAJS4cB5-xLEib-8GBMg ""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36""


In [3]:
import praw
import re

# Initialize the Reddit instance with your Reddit API credentials
reddit = praw.Reddit(
    client_id = os.getenv("REDDIT_CLIENT_ID"),
    client_secret = os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent = os.getenv("REDDIT_USER_AGENT")
)

subreddit_name = 'personalfinance'  # The subreddit you're interested in

def fetch_links(wiki_page_title, visited_pages=set()):
    internal_links = []
    try:
        # Fetch the wiki page content
        wiki_page = reddit.subreddit(subreddit_name).wiki[wiki_page_title]
        content = wiki_page.content_md
        # Extract all links from the content
        links = re.findall(r'\[.*?\]\((.*?)\)', content)
        # Filter out external links and prepare for recursion
        for link in links:
            if 'reddit.com/r/' + subreddit_name + '/wiki/' in link:
                internal_link = link.split('/wiki/')[-1]
                if internal_link not in visited_pages:
                    visited_pages.add(internal_link)
                    internal_links.append(link)
                    internal_links.extend(fetch_links(internal_link, visited_pages))
    except Exception as e:
        print(f"Error fetching page '{wiki_page_title}': {str(e)}")
    return list(set(internal_links))  # Return a list of unique links

# Start the process from the main wiki page or any specific page you're interested in
initial_page = 'index'  # Use 'index' for the main wiki index page
all_links = fetch_links(initial_page)
print(f"Collected {len(all_links)} unique internal wiki links:")
for link in all_links:
    print(link)

Error fetching page 'credit\_reports': INVALID_PAGE_NAME
Error fetching page 'credit\_building': INVALID_PAGE_NAME
Collected 99 unique internal wiki links:
https://www.reddit.com/r/personalfinance/wiki/employment
https://www.reddit.com/r/personalfinance/wiki/retirementaccounts/rollovers
https://www.reddit.com/r/personalfinance/wiki/charity
https://www.reddit.com/r/personalfinance/wiki/investing
https://www.reddit.com/r/personalfinance/wiki/pastdiscussions
https://www.reddit.com/r/personalfinance/wiki/studentloans
https://www.reddit.com/r/personalfinance/wiki/umbrella_insurance
https://www.reddit.com/r/personalfinance/wiki/snoonet
https://www.reddit.com/r/personalfinance/wiki/teachme_engineering
https://www.reddit.com/r/personalfinance/wiki/jointfinances
https://www.reddit.com/r/personalfinance/wiki/kicked_out
https://www.reddit.com/r/personalfinance/wiki/kicked_out#wiki_financial_accounts
https://www.reddit.com/r/personalfinance/wiki/teachme
https://old.reddit.com/r/personalfinance/wik

In [8]:
%pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [10]:
import requests
from bs4 import BeautifulSoup
import re
import time
import os

# List of URLs to extract text from
urls = all_links  # Replace <your_list_of_urls_here> with your list of URLs

def extract_relevant_text_from_url(url, retries=5, backoff_factor=1.0):
    for attempt in range(retries):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                # Logic to find the main content goes here as before
                
                main_content = soup.find('main') or soup.find('article') or soup.find('div', role='main')
                if not main_content:
                    main_content = soup.body
                text = ' '.join(t.get_text(separator='\n', strip=True) for t in main_content.find_all(recursive=False) if t.name not in ['script', 'style'])
                
                # Check if the extracted text indicates a failure to load the content properly
                if "An unknown error occurred" not in text:
                    return text
                else:
                    # If the specific error text is found, raise an exception to trigger a retry
                    raise ValueError("Content indicates an error page")
            else:
                # Non-200 responses are treated as errors and will trigger a retry
                raise ValueError(f"HTTP error {response.status_code}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(backoff_factor * (2 ** attempt))  # Exponential backoff

    return "Failed to retrieve content after several attempts"

# Function to save text to a file
def save_text_to_file(filename, text):
    # Define the base directory where files should be saved
    base_dir = "/Users/mrinoyb2/git/FinanceGPT/data/pf_wiki_texts"
    # Construct the full path for the file to be saved
    full_path = os.path.join(base_dir, filename)
    with open(full_path, 'w', encoding='utf-8') as file:
        file.write(text)

# Main loop to process each URL
for url in urls:
    print(url)
    text = extract_relevant_text_from_url(url)
    # Creating a filename from the URL. Filename should be the last part of the URL
    filename = re.sub(r'\W+', '', url.split('/')[-1]) + '.txt'
    save_text_to_file(filename, text)
    print(f"Saved content from {url} to {filename}")


https://www.reddit.com/r/personalfinance/wiki/end_of_life_planning
Saved content from https://www.reddit.com/r/personalfinance/wiki/end_of_life_planning to end_of_life_planning.txt
https://www.reddit.com/r/personalfinance/wiki/401k_funds
Attempt 1 failed: Content indicates an error page
Attempt 2 failed: Content indicates an error page
Saved content from https://www.reddit.com/r/personalfinance/wiki/401k_funds to 401k_funds.txt
https://www.reddit.com/r/personalfinance/wiki/psychology
Saved content from https://www.reddit.com/r/personalfinance/wiki/psychology to psychology.txt
https://www.reddit.com/r/personalfinance/wiki/financialinstitutions
Saved content from https://www.reddit.com/r/personalfinance/wiki/financialinstitutions to financialinstitutions.txt
https://www.reddit.com/r/personalfinance/wiki/retirement_predicaments
Attempt 1 failed: Content indicates an error page
Attempt 2 failed: Content indicates an error page
Saved content from https://www.reddit.com/r/personalfinance/wik